In [1]:
#!pip install featurewiz
#!pip install xlrd
#!pip install Pillow==9.0.0
#!pip install autokeras
#!pip install auto-sklearn

Import packages

In [2]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from featurewiz import featurewiz
import tensorflow as tf
import autokeras as ak

np.random.seed(1234)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Imported version = 0.1.60.
from featurewiz import FeatureWiz
wiz = FeatureWiz(verbose=1)
X_train_selected = wiz.fit_transform(X_train, y_train)
X_test_selected = wiz.transform(X_test)
wiz.features  ### provides a list of selected features ###
                                


In [3]:
def over_sample_train_test(x, y):
    oversample = SMOTE()
    x, y = oversample.fit_resample(x, y)
    return x, y

def smoth_train_test(x, y):
    ros = RandomOverSampler(random_state=0)
    ros.fit(x, y)
    x_res, y_res = ros.fit_resample(x, y)
    return x_res, y_res

def dummy_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation):
    dclf = DummyClassifier()
    dclf.fit(x_train, y_train)
    print('ZeroR: T: ', dclf.score(x_test, y_test))
    print('ZeroR: V: ', dclf.score(x_validation, y_validation))

def knn_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation):
    clf = KNeighborsClassifier(n_neighbors=2)
    clf.fit(x_train, y_train)
    print('Knn: T: ', clf.score(x_test, y_test))
    print('Knn: V: ', clf.score(x_validation, y_validation))
    
def xgboost_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation):
    xgb = XGBClassifier(use_label_encoder=False, random_state=42, eval_metric='mlogloss')
    xgb.fit(x_train, y_train)
    print('Xgb: T: ', xgb.score(x_test, y_test))
    print('Xgb: V: ', xgb.score(x_validation, y_validation))

Load dataset

In [4]:
df_train = pd.read_csv('drive/MyDrive/Colab Notebooks/Detecting_Anomalies/Train.csv')
df_x = df_train.drop("Class", axis = 1)
df_y = df_train["Class"]

In [5]:
x_train, x_validation, y_train, y_validation = train_test_split(
    df_x, df_y, test_size=0.1, stratify = df_y)

x_over, y_over = over_sample_train_test(x_train, y_train)

x_train, x_test, y_train, y_test = train_test_split(
    x_over, y_over, test_size=0.2, stratify = y_over)

dummy_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)
knn_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)
xgboost_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)

ZeroR: T:  0.49914236706689535
ZeroR: V:  0.07909604519774012
Knn: T:  0.902229845626072
Knn: V:  0.8531073446327684
Xgb: T:  0.9262435677530018
Xgb: V:  0.8757062146892656


In [6]:
x_train, x_validation, y_train, y_validation = train_test_split(
    df_x, df_y, test_size=0.1, stratify = df_y)

x_over, y_over = smoth_train_test(x_train, y_train)

x_train, x_test, y_train, y_test = train_test_split(
    x_over, y_over, test_size=0.2, stratify = y_over)

dummy_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)
knn_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)
xgboost_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)

ZeroR: T:  0.49914236706689535
ZeroR: V:  0.9209039548022598
Knn: T:  0.9554030874785592
Knn: V:  0.9152542372881356
Xgb: T:  0.9588336192109777
Xgb: V:  0.9152542372881356


In [7]:
#target = 'Class'
 
#features, train = featurewiz(df_train, target, corr_limit=0.7, verbose=2, sep=",",
#header=0, test_data="", feature_engg="", category_encoders="")
#dfx_x = train.drop("Class", axis = 1)
#dfx_y = train["Class"]

#x_train, x_validation, y_train, y_validation = train_test_split(
#    dfx_x, dfx_y, test_size=0.1, stratify = df_y)

#x_train, x_test, y_train, y_test = smoth_train_test(x_train, y_train)
#dummy_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)
#knn_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)
#xgboost_classifier(x_train, x_test, y_train, y_test, x_validation, y_validation)

In [ ]:
x_train, x_validation, y_train, y_validation = train_test_split(
    df_x, df_y, test_size=0.1, stratify = df_y)

x_over, y_over = smoth_train_test(x_train, y_train)

# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    # The path to the train.csv file.
    x_over,
    # The name of the label column.
    y_over,
    epochs = 10,
)
# Predict with the best model.
predicted_y = clf.predict(x_validation)
# Evaluate the best model with testing data.
print(clf.evaluate(x_validation, y_validation))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
True              |?                 |structured_data_block_1/normalize
2                 |?                 |structured_data_block_1/dense_block_1/num_layers
False             |?                 |structured_data_block_1/dense_block_1/use_batchnorm
0                 |?                 |structured_data_block_1/dense_block_1/dropout
32                |?                 |structured_data_block_1/dense_block_1/units_0
32                |?                 |structured_data_block_1/dense_block_1/units_1
0                 |?                 |classification_head_1/dropout
adam              |?                 |optimizer
0.001             |?                 |learning_rate

